In [1]:
import numpy as np

###### 读写文件
创建一个单位矩阵并将其存储到文件中

In [2]:
i2 = np.eye(2) # 创建一个二维单位矩阵数组
print(i2)
np.savetxt('eye.txt',i2) # 用savetxt函数进行写文件的操作

[[ 1.  0.]
 [ 0.  1.]]


###### 读入CSV文件
载入苹果公司的历史股价数据为例。股价数据存储在CSV中，第一列为股票代码，第二列为dd-mm-yyyy格式日期，第三列为空，随后各列依次为开盘价、最高价、最低价和收盘价，最后一列为当日的成交量。

In [3]:
# 用loadtxt函数读取csv文件，用delimiter参数指定文件中的分隔符，用usecols参数指定要导入的数据列，将unpack参数设置为True使不同列的数据分开存储
c, v = np.loadtxt('data.csv',delimiter=',', usecols=(6,7), unpack=True)

###### 成交量加权平均价格（VWAP）
VWAP是一个非常重要的经济学量，代表着金融资产的“平均”价格。某个价格的成交量越高，该价格所占的权重就越大。VWAP就是以成交量为权重计算出来的加权平均值，常用于算法交易。

In [5]:
vmap = np.average(c, weights=v) 
print("VWAP = " + str(vmap))

VWAP = 350.589549353


###### 算术平均值

In [6]:
mean = np.mean(c)
print ("算术平均值mean = " + str(mean))

算术平均值mean = 351.037666667


###### 时间加权平均价格（TWAP）
在经济学中，TWAP是另一种“平均”价格的指标。基本思想就是最近的价格重要性大一些，所以应对近期的价格给以较高的权重。最简单的方法是用arange函数创建一个从0开始依次增长的自然数序列，自然数的个数即为收盘价的个数。这并不一定是正确的计算TWAP的方式，示例仅仅是为了说明问题。

In [7]:
t = np.arange(len(c))
twap = np.average(c, weights=t)
print("TWAP = " + str(twap))

TWAP = 352.428321839


###### 找到最大值和最小值

In [9]:
# 载入每日最高价和最低价的数据
h, l = np.loadtxt('data.csv', delimiter=',', usecols=(4,5), unpack=True)
# 获取价格区间
highest = np.max(h)
lowest = np.min(l)
print("最高价格highest =  " + str(highest))
print("最低价格lowest =  " + str(lowest))

最高价格highest =  364.9
最低价格lowest =  333.53


In [10]:
# ptp函数计算数组的取值范围，返回的是数组元素的最大值和最小值之间的差值
spread_high_price = np.ptp(h)
spread_low_price = np.ptp(l)
print("最高价之间的差值spread_high_price =  " + str(spread_high_price))
print("最低价之间的差值spread_low_price =  " + str(spread_low_price))

最高价之间的差值spread_high_price =  24.86
最低价之间的差值spread_low_price =  26.97


###### 简单统计分析

In [11]:
# 调用median函数计算收盘价的中位数
median = np.median(c)
print("收盘价的中位数median = " + str(median))

收盘价的中位数median = 352.055


数学上常规方法计算中位数：<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;首先对数组进行从小到大排序；<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;如果数组长度为奇数，中位数就是中间位的数；<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;如果数组长度为偶数，中位数的值等于中间那两个数的平均值。

In [14]:
# 对价格数组进行排序
sorted_c = np.msort(c)
print("排序后的价格数组sorted = " + str(sorted_c))

排序后的价格数组sorted = [ 336.1   338.61  339.32  342.62  342.88  343.44  344.32  345.03  346.5
  346.67  348.16  349.31  350.56  351.88  351.99  352.12  352.47  353.21
  354.54  355.2   355.36  355.76  356.85  358.16  358.3   359.18  359.56
  359.9   360.    363.13]


In [15]:
# 根据中位数定义求中位数
N = len(c)
if N % 2 == 1:   # 如果长度为奇数
    middle = sorted_c[(N - 1)/2]
else:   # 长度为偶数
    middle = (sorted_c[N / 2] + sorted_c[(N-1) / 2]) / 2
print("中位数middle = " + str(middle))

中位数middle = 352.055


方差：能体现变量变化的程度。在示例中，方差体现为投资风险的大小。<br>
numpy文档中方差的定义：各个数据与所有数据算术平方数的离差平方和除以数据个数所得到的值。<br>
一般统计学书上样本方差的定义：离差平方和/(n-1) ,其中n为样本个数。<br>
之所以有这样的差别，是为了保证样本方差是一个无偏估计量，但numpy文档中方差的定义在机器学习领域更为通用。

In [17]:
# 根据方差函数计算方差
variance = np.var(c)
print("根据函数计算收盘价的方差variance = " + str(variance))
# 根据定义计算方差
variance_from_definition = np.mean((c - np.mean(c)) ** 2)
print("根据定义计算收盘价的方差variance_from_definition = " + str(variance_from_definition))

根据函数计算收盘价的方差variance = 50.1265178889
根据定义计算收盘价的方差variance_from_definition = 50.1265178889


###### 股票收益率
在学术文献中，收盘价的分析常常是基于股票收益率和对数收益率的。<br>
简单收益率是指相邻两个价格之间的变化率。<br>
对数收益率是指所有价格取对数后两两之间的差值。对数收益率也可以用来衡量价格的变化率。<br>
收益率是无量纲的。投资者感兴趣的是收益率的方差或标准差，因为这代表这投资风险的大小。

In [18]:
# 计算简单收益率。
# diff函数返回一个由相邻数组元素的差值构成的数组
returns = np.diff(c) / c[:-1]
# 用std函数计算标准差
standard_deviation = np.std(returns)
print("简单收益率标准差standard_deviation = " + str(standard_deviation))

简单收益率标准差standard_deviation = 0.0129221344368


In [19]:
# 计算对数收益率
logreturns = np.diff(np.log(c)) # 一般情况下需要检查输入数组以确保其不含零和负数，在该例中股价总为正值，可省略检查。
# 我们对哪些交易日的收益率为正值感兴趣
posretindices = np.where(returns > 0) # where函数可以根据指定的条件返回所有满足条件的数组元素的索引值。
print("收益率为正的索引： " + str(posretindices))

收益率为正的索引： (array([ 0,  1,  4,  5,  6,  7,  9, 10, 11, 12, 16, 17, 18, 19, 21, 22, 23,
       25, 28], dtype=int64),)


波动率是对价格变动的一种度量。年波动率等于对数收益率的标准差除以其均值，再除以交易日倒数的平方根，通常交易日取252天。

In [20]:
# 计算波动率
annual_volatility = np.std(logreturns) / np.mean(logreturns)
annual_volatility = annual_volatility / np.sqrt(1. / 252.)
print("年波动率annual_volatility = " + str(annual_volatility))
monthly_volatility = annual_volatility * np.sqrt(1. / 12.)
print("月波动率monthly_volatility = " + str(monthly_volatility))

年波动率annual_volatility = 129.274789911
月波动率monthly_volatility = 37.3184173773


###### 日期分析
Numpy是面向浮点数运算的，因此需要对日期做一些专门的处理。我们需要显式地告诉Numpy怎样来转换日期，需要用到loadtxt函数中converters参数，该参数是数据列和转换函数之间进行映射的字典。<br>
写出转换函数：<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;星期一   0<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;星期二   1<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;星期三   2<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;星期四   3<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;星期五   4<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;星期六   5<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;星期日   6<br>

In [23]:
import datetime
# 将日期作为字符串传给datestr2num函数，按照指定的形式"%d-%m-%Y"转换成一个datetime对象，
# 随后datetime对象被转换为date对象，最后调用weekday方法返回一个数字
def datestr2num(s):
    return datetime.datetime.strptime(s, "%d-%m-%Y").date().weekday()

In [24]:
# 将日期转换函数挂接上去，读入数据
dates = np.loadtxt('data.csv', delimiter=',', usecols=(1,), converters={1:datestr2num}, unpack=True)
print("日期Dates = " + str(dates))

日期Dates = [ 4.  0.  1.  2.  3.  4.  0.  1.  2.  3.  4.  0.  1.  2.  3.  4.  1.  2.
  3.  4.  0.  1.  2.  3.  4.  0.  1.  2.  3.  4.]


In [25]:
# 创建一个包含5个元素的数组，分别代表一周的5个工作日，数组初始化为0，这个数组将用于保存各工作日的平均收盘价。
averages = np.zeros(5)
# 遍历星期一到星期五
for i in range(5):
    indices = np.where(dates == i)  # 用where函数得到各工作日的索引值并存储在indices数组中
    prices = np.take(c,indices)     # 再用take函数获取这些索引值相应的元素值
    avg = np.mean(prices)
    print("Day "+str(i)+" prices "+str(prices)+" Average "+str(avg))
    averages[i] = avg  # 对每个工作日计算出的平均值存放在averages数组中

Day 0 prices [[ 339.32  351.88  359.18  353.21  355.36]] Average 351.79
Day 1 prices [[ 345.03  355.2   359.9   338.61  349.31  355.76]] Average 350.635
Day 2 prices [[ 344.32  358.16  363.13  342.62  352.12  352.47]] Average 352.136666667
Day 3 prices [[ 343.44  354.54  358.3   342.88  359.56  346.67]] Average 350.898333333
Day 4 prices [[ 336.1   346.5   356.85  350.56  348.16  360.    351.99]] Average 350.022857143


In [26]:
# 找出哪个工作日的平均收盘价最高，哪个最低
top = np.max(averages)
print("最高工作日平均收盘价top = " + str(top))
print("平均收盘价最高的工作日是： " + str(np.argmax(averages)))   # argmax函数返回的是averages数组中最大元素的索引值
bottom = np.min(averages)
print("最低工作日平均收盘价bottom = " + str(bottom))
print("平均收盘价最低的工作日是： " + str(np.argmin(averages)))   # argmin函数返回的是averages数组中最小元素的索引值

最高工作日平均收盘价top = 352.136666667
平均收盘价最高的工作日是： 2
最低工作日平均收盘价bottom = 350.022857143
平均收盘价最低的工作日是： 4


###### 汇总数据
汇总整个交易周中从周一到周五的所有数据。

In [32]:
# 只考虑前三周的数据，避免节假日造成的数据缺失。
c = c[:16]
dates = dates[:16]
# 找到第一个星期一
first_monday = np.ravel(np.where(dates == 0))[0]
print('第一个星期一的索引：' + str(first_monday))

第一个星期一的索引：1


In [35]:
# 找到最后一个星期五
last_friday = np.ravel(np.where(dates == 4))[-1]
print('最后一个星期五的索引：' + str(last_friday))

最后一个星期五的索引：15


In [40]:
# 创建一个数组，用于存储三周内每一天的索引值
weeks_indices = np.arange(first_monday, last_friday +1)
print("周索引初始化： " + str(weeks_indices))

周索引初始化： [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


In [41]:
# 按照每个子数组5个元素，用split函数切分数组
weeks_indices = np.split(weeks_indices, 3)
print("分组之后的周索引： " + str(weeks_indices))

分组之后的周索引： [array([1, 2, 3, 4, 5], dtype=int64), array([ 6,  7,  8,  9, 10], dtype=int64), array([11, 12, 13, 14, 15], dtype=int64)]
